In [ ]:
import yaml
import logging

def load_credentials(filepath):
    try:
        with open(filepath, 'r') as file:
            credentials = yaml.safe_load(file)
            email = credentials['EMAIL_ACCOUNT']
            app_password = credentials['APP_PASSWORD']
            server = credentials['IMAP_SERVER']
            return email, app_password, server
    except Exception as e:
        logging.error("Failed to load credentials: {}".format(e))
        raise
    
EMAIL_ACCOUNT, APP_PASSWORD, IMAP_SERVER = load_credentials("creds.yaml")

### Using IMAP

In [ ]:
import imaplib
import email
from email.header import decode_header
import pandas as pd

def connect_to_mailbox(mailbox_name):
    """Login and select the desired mailbox."""
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL_ACCOUNT, APP_PASSWORD)
    mail.select(mailbox_name)
    return mail

def decode_header_value(value):
    """Safely decode email headers."""
    if not value:
        return ""
    parts = decode_header(value)
    decoded = ''
    for part, encoding in parts:
        if isinstance(part, bytes):
            decoded += part.decode(encoding if encoding else 'utf-8', errors='ignore')
        else:
            decoded += part
    return decoded

def fetch_last_n_emails(mail, n=50):
    """Fetch last N email headers and content from the selected mailbox."""
    status, messages = mail.search(None, 'ALL')
    if status != 'OK':
        print("Failed to fetch emails.")
        return pd.DataFrame()

    email_ids = messages[0].split()
    last_n_ids = email_ids[-n:]

    emails = []
    for uid in reversed(last_n_ids):
        status, data = mail.fetch(uid, '(RFC822)')  # Fetch full message
        if status != 'OK':
            continue

        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Extract content
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))

                # Ignore attachments
                if "attachment" in content_disposition:
                    continue

                if content_type == "text/plain":
                    body = part.get_payload(decode=True).decode(errors="ignore")
                    break
        else:
            body = msg.get_payload(decode=True).decode(errors="ignore")

        email_data = {
            'from': decode_header_value(msg.get("From")),
            'to': decode_header_value(msg.get("To")),
            'subject': decode_header_value(msg.get("Subject")),
            'date': msg.get("Date"),
            'cc': decode_header_value(msg.get("Cc")),
            'bcc': decode_header_value(msg.get("Bcc")),
            'reply_to': decode_header_value(msg.get("Reply-To")),
            'message_id': msg.get("Message-ID"),
            'body': body.strip()
        }

        emails.append(email_data)

    return pd.DataFrame(emails)

def display_emails(df, section_name):
    """Print a preview of the DataFrame."""
    print(f"\n=== {section_name.upper()} SECTION ===")
    print(df[['from', 'subject', 'date']].head())

def main():
    # Fetch from Primary
    primary_mail = connect_to_mailbox('INBOX')
    df_primary = fetch_last_n_emails(primary_mail, n=50)
    primary_mail.logout()
    return df_primary

emails = main()
emails